In [1]:
import os

In [2]:
%pwd

'f:\\PROJECTS\\imdb 50k NLP\\Sentiment-analysis-mlflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\PROJECTS\\imdb 50k NLP\\Sentiment-analysis-mlflow'

In [6]:
os.environ['MLFLOW_TRACKING_URI']= 'https://dagshub.com/chrisaaryan/my-first-repo.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']= 'chrisaaryan'
os.environ['MLFLOW_TRACKING_PASSWORD']= 'Github34chris@17'